# Scattering resonances for a circular metamaterial cavity

The codes below are associated to the article:

- C. Carvalho and Z. Moitier, _Asymptotics for metamaterial cavities and their effect on scattering_ [[arXiv](https://arxiv.org/abs/2010.07583), [HAL](https://hal.archives-ouvertes.fr/hal-02965993)]

We present computations related to Section 3 and Appendix C of the manuscript.

## Zoïs Moitier, Camille Carvalho (2021)
            
_Karlsruhe Institute of Technology, Germany_

_University of California Merced, USA_

In [1]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import Normalize, SymLogNorm
from scipy.special import hankel1, iv

import src

%matplotlib inline

## Internal functions

In [2]:
def plot_scatter(ax, m, λ, M, lw, marker, zorder, label):
    sc = ax.scatter(
        np.real(λ),
        np.imag(λ),
        s=15,
        lw=lw,
        c=m,
        marker=marker,
        zorder=zorder,
        vmin=0,
        vmax=M,
        label=label,
    )
    return sc

In [3]:
def plot_λ(ε, M, ax, xlim, ylim):
    η = np.sqrt(-ε)

    data = np.load(f"data/eps_{ε}.npz")
    m_out, λ_out = src.parse_data_λ(data, "outer", M)
    m_inn, λ_inn = src.parse_data_λ(data, "inner", M)
    m_pla, λ_pla = src.parse_data_λ(data, "plasmon", M)

    _ = plot_scatter(ax, m_out, λ_out, M, 0.5, "v", 2, r"$\mathcal{R}_{\mathsf{out}}$")
    sc_inn = plot_scatter(
        ax, m_inn, λ_inn, M, 0.2, ".", 3, r"$\mathcal{R}_{\mathsf{inn}}$"
    )
    _ = plot_scatter(ax, m_pla, λ_pla, M, 1, "+", 4, r"$\mathcal{R}_{\mathsf{pla}}$")

    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)

    ax.set_xlabel(r"$\Re(\ell^2)$")
    ax.set_ylabel(r"$\Im(\ell^2)$")

    ax.grid(True, zorder=1)
    ax.legend(
        loc="lower center",
        bbox_to_anchor=(0.5, 1.025),
        ncol=3,
        borderaxespad=0.0,
        handlelength=0,
        markerscale=2,
    )
    ax.set_title(f"ε = {ε}", y=1.125)

    return sc_inn

In [4]:
def plot_resonances(M, ε1, ε2, xmin, xmax, ymin, ymax):
    if src.is_number(xmin) and src.is_number(xmax):
        xlim = (float(xmin), float(xmax))
    else:
        xlim = None

    if src.is_number(ymin) and src.is_number(ymax):
        ylim = (float(ymin), float(ymax))
    else:
        ylim = None

    plt.rcParams.update(src.set_rcParams(font_size=15))
    fig, ax = plt.subplots(
        ncols=2,
        figsize=src.set_size(frac_width=2, frac_height=0.5),
        constrained_layout=True,
    )

    sc1 = plot_λ(ε1, M, ax[0], xlim=xlim, ylim=ylim)
    sc2 = plot_λ(ε2, M, ax[1], xlim=xlim, ylim=ylim)

    cbar = fig.colorbar(sc1, ax=ax[0])
    cbar.set_label(r"$m$")

    cbar = fig.colorbar(sc2, ax=ax[1])
    cbar.set_label(r"$m$")

    plt.show()

    return None

In [5]:
def calc_mode(η, m, k, R, T, N, Int, Ext):
    U = np.empty((N, N), dtype=complex)
    U[Int] = iv(m, η * k * R[Int]) / iv(m, η * k)
    U[Ext] = hankel1(m, k * R[Ext]) / hankel1(m, k)
    U *= np.exp(1j * m * T)
    return U

In [6]:
def plot_mode(fig, ax, X, Y, U, name, Sym_Log_Norm=False):
    u_max = np.abs(U).max()

    if Sym_Log_Norm:
        norm = SymLogNorm(1, vmin=-u_max, vmax=u_max, base=10)
    else:
        norm = Normalize(vmin=-u_max, vmax=u_max)

    im = ax.pcolormesh(X, Y, np.real(U), shading="gouraud", cmap="RdBu_r", norm=norm)
    ax.add_artist(plt.Circle((0, 0), 1, fill=False, ec="k", lw=2, ls=":"))

    ax.set_aspect("equal", "box")
    ax.set_title(name)
    fig.colorbar(im, ax=ax)

    return None

In [7]:
def plot_modes(η, m, k_out, q_out, k_inn, q_inn, k_pla):
    plt.rcParams.update(src.set_rcParams(font_size=15))
    fig, ax = plt.subplots(
        ncols=3,
        figsize=src.set_size(frac_width=2.5, frac_height=0.25),
        constrained_layout=True,
    )

    h = np.linspace(-H, H, num=N)
    X, Y = np.meshgrid(h, h)
    R, T = np.hypot(X, Y), np.arctan2(Y, X)
    Int, Ext = np.where(np.less(R, 1)), np.where(np.greater_equal(R, 1))

    U_out = calc_mode(η, m, k_out[q_out], R, T, N, Int, Ext)
    plot_mode(
        fig, ax[0], X, Y, U_out, f"outer, k = {k_out[q_out]:.1e}", Sym_Log_Norm=True
    )

    U_inn = calc_mode(η, m, k_inn[q_inn], R, T, N, Int, Ext)
    plot_mode(fig, ax[1], X, Y, U_inn, f"inner, k = {k_inn[q_inn]:.1e}")

    if m > 0:
        U_pla = calc_mode(η, m, k_pla[0], R, T, N, Int, Ext)
        plot_mode(fig, ax[2], X, Y, U_pla, f"plasmon, k = {k_pla[0]:.1e}")

    return None

In [8]:
def plot_wavenumber(ε, m, H, N):
    η = np.sqrt(-ε)

    data = np.load(f"data/eps_{ε}.npz")
    m_out, k_out = src.parse_data_k(data, "outer", m)
    m_inn, k_inn = src.parse_data_k(data, "inner", m)
    m_pla, k_pla = src.parse_data_k(data, "plasmon", m)

    plt.rcParams.update(src.set_rcParams(font_size=15))
    fig, ax = plt.subplots(
        figsize=src.set_size(frac_width=1.25), constrained_layout=True
    )

    ax.plot(np.real(k_out), np.imag(k_out), "C0v", label=r"$k_{\mathsf{out}}$")
    ax.plot(np.real(k_inn), np.imag(k_inn), "C1o", label=r"$k_{\mathsf{inn}}$")
    ax.plot(np.real(k_pla), np.imag(k_pla), "C2P", label=r"$k_{\mathsf{pla}}$")

    xmin, xmax = ax.get_xlim()
    if abs(xmax) < 1e-5:
        xmax = 1
        ax.set_xlim(-1, 1)

    for i, k in enumerate(k_out):
        plt.text(
            k.real + 0.015 * xmax,
            k.imag,
            f"{i}",
            ha="left",
            va="center",
            color="C0",
            fontsize=13,
        )
    for i, k in enumerate(k_inn):
        plt.text(
            k.real + 0.015 * xmax,
            k.imag,
            f"{i}",
            ha="left",
            va="center",
            color="C1",
            fontsize=13,
        )

    ax.set_xlabel(r"$\Re(\ell)$")
    ax.set_ylabel(r"$\Im(\ell)$")

    ax.grid(True)
    ax.legend()
    ax.set_title(f"ε = {ε}")

    q_out = widgets.IntSlider(
        value=0, min=0, max=np.size(k_out, 0) - 1, step=1, description="q_out"
    )
    q_out.style.handle_color = "#1F77B4"
    q_inn = widgets.IntSlider(
        value=0, min=0, max=np.size(k_inn, 0) - 1, step=1, description="q_inn"
    )
    q_inn.style.handle_color = "#FF7F0E"

    ui = widgets.HBox([q_out, q_inn])
    out = widgets.interactive_output(
        plot_modes,
        {
            "η": widgets.fixed(η),
            "m": widgets.fixed(m),
            "k_out": widgets.fixed(k_out),
            "q_out": q_out,
            "k_inn": widgets.fixed(k_inn),
            "q_inn": q_inn,
            "k_pla": widgets.fixed(k_pla),
        },
    )

    display(ui, out)

    return None

## Scattering resonances

In [9]:
εcav = [-1.5, -1.3, -1.2, -1.1, -0.9, -0.8, -0.75, -0.7]

grid = widgets.GridspecLayout(2, 4)
grid[0, 0] = widgets.IntSlider(value=64, min=1, max=64, step=1, description="M")
grid[0, 1] = widgets.Dropdown(options=εcav, value=-1.1, description="ε left")
grid[0, 2] = widgets.Dropdown(options=εcav, value=-0.9, description="ε right")
grid[1, 0] = widgets.FloatText(value=-1750, description="xmin")
grid[1, 1] = widgets.FloatText(value=750, description="xmax")
grid[1, 2] = widgets.FloatText(value=-700, description="ymin")
grid[1, 3] = widgets.FloatText(value=35, description="ymax")

out = widgets.interactive_output(
    plot_resonances,
    {
        "M": grid[0, 0],
        "ε1": grid[0, 1],
        "ε2": grid[0, 2],
        "xmin": grid[1, 0],
        "xmax": grid[1, 1],
        "ymin": grid[1, 2],
        "ymax": grid[1, 3],
    },
)

display(grid, out)

GridspecLayout(children=(IntSlider(value=64, description='M', layout=Layout(grid_area='widget001'), max=64, mi…

Output()

## Modes

In [10]:
m = widgets.IntSlider(value=8, min=0, max=32, step=1, description="m")
ε = widgets.Dropdown(options=εcav, value=-1.1, description="ε")
H = 1.25
N = 128

ui = widgets.HBox([m, ε])
out = widgets.interactive_output(
    plot_wavenumber, {"m": m, "ε": ε, "H": widgets.fixed(H), "N": widgets.fixed(N)}
)

display(ui, out)

Output()